In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# Hyper Parameters
num_epochs = 5
learning_rate = 0.001

Now, CIFAR10 training and test dataset is loaded. Also the PILImage is transforms to tensors in the normalized range of [-1,1]

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Now, Convolution Neural Network is defined. 

In [3]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


cnn = CNN()

For loss function and optimizer, I am goiing to use Classification Cross-Entropy loss and SGD with momentum



In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=learning_rate, momentum=0.9)

Training the network for 5 passes over the entrire training dataset. 

In [5]:
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)

        
        optimizer.zero_grad()   # zero the parameter gradients

        # Forward + backward + Optimize
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics 
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batclearning_ratehes
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.272
[1,  4000] loss: 1.891
[1,  6000] loss: 1.690
[1,  8000] loss: 1.584
[1, 10000] loss: 1.531
[1, 12000] loss: 1.480
[2,  2000] loss: 1.418
[2,  4000] loss: 1.368
[2,  6000] loss: 1.350
[2,  8000] loss: 1.357
[2, 10000] loss: 1.324
[2, 12000] loss: 1.312
[3,  2000] loss: 1.234
[3,  4000] loss: 1.236
[3,  6000] loss: 1.248
[3,  8000] loss: 1.218
[3, 10000] loss: 1.222
[3, 12000] loss: 1.209
[4,  2000] loss: 1.155
[4,  4000] loss: 1.136
[4,  6000] loss: 1.148
[4,  8000] loss: 1.125
[4, 10000] loss: 1.129
[4, 12000] loss: 1.131
[5,  2000] loss: 1.061
[5,  4000] loss: 1.062
[5,  6000] loss: 1.069
[5,  8000] loss: 1.073
[5, 10000] loss: 1.077
[5, 12000] loss: 1.076
Finished Training


Testing the CNN

In [7]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = cnn(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 59 %


Performance of individual classes

In [9]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs = cnn(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 62 %
Accuracy of   car : 70 %
Accuracy of  bird : 47 %
Accuracy of   cat : 19 %
Accuracy of  deer : 48 %
Accuracy of   dog : 55 %
Accuracy of  frog : 82 %
Accuracy of horse : 65 %
Accuracy of  ship : 82 %
Accuracy of truck : 60 %
